In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master("spark://spark-master:7077").config("spark.jars.packages", 
                                                                        "org.apache.hadoop:hadoop-aws:3.3.4").appName("spark-hw").getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fc5d0391-1c56-4a12-9d48-3eb281459b96;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 156ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------------------

In [3]:
print(spark.version)

3.5.5


In [4]:
actor_df = spark.read.csv('../data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('../data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('../data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('../data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('../data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('../data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('../data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('../data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('../data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('../data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('../data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('../data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('../data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('../data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('../data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
df_films_count = (
    film_category_df
    .join(F.broadcast(category_df.select("category_id", "name")), on="category_id", how="inner")
    .groupBy("category_id", "name")
    .agg(F.count("film_id").alias("film_count"))
    .orderBy(F.col("film_count").desc())
    .drop("category_id")
)

df_films_count.show()

+-----------+----------+
|       name|film_count|
+-----------+----------+
|     Sports|        74|
|    Foreign|        73|
|     Family|        69|
|Documentary|        68|
|  Animation|        66|
|     Action|        64|
|        New|        63|
|      Drama|        62|
|     Sci-Fi|        61|
|      Games|        61|
|   Children|        60|
|     Comedy|        58|
|   Classics|        57|
|     Travel|        57|
|     Horror|        56|
|      Music|        51|
+-----------+----------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [6]:
top_actors_df = (
    rental_df[["rental_id", "inventory_id"]]
    .join(inventory_df[["film_id", "inventory_id"]], on="inventory_id", how="inner")
    .join(film_actor_df[["film_id", "actor_id"]], on="film_id", how="inner")
    .join(F.broadcast(actor_df[["actor_id", "first_name", "last_name"]]), on="actor_id",  how="inner")
    .groupBy("actor_id", "first_name", "last_name")
    .agg(F.count("film_id").alias("film_count"))
    .orderBy(F.col("film_count").desc())
    .limit(10)
)

top_actors_df.show()

+--------+----------+-----------+----------+
|actor_id|first_name|  last_name|film_count|
+--------+----------+-----------+----------+
|     107|      GINA|  DEGENERES|       753|
|     181|   MATTHEW|     CARREY|       678|
|     198|      MARY|     KEITEL|       674|
|     144|    ANGELA|WITHERSPOON|       654|
|     102|    WALTER|       TORN|       640|
|      60|     HENRY|      BERRY|       612|
|     150|     JAYNE|      NOLTE|       611|
|      37|       VAL|     BOLGER|       605|
|      23|    SANDRA|     KILMER|       604|
|      90|      SEAN|    GUINESS|       599|
+--------+----------+-----------+----------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [7]:
top_category_df = (
    payment_df[["rental_id", "amount"]]
    .join(rental_df[["rental_id", "inventory_id"]], on="rental_id", how="inner")
    .join(inventory_df[["film_id", "inventory_id"]], on="inventory_id", how="inner")
    .join(film_category_df[["film_id", "category_id"]], on="film_id", how="inner")
    .join(F.broadcast(category_df[["category_id", "name"]]), on="category_id",  how="inner")
    .groupBy("name")
    .agg(F.bround(F.sum("amount"), 2).alias("total_amount"))
    .orderBy(F.col("total_amount").desc())
    .limit(1)
)

top_category_df.show()

+------+------------+
|  name|total_amount|
+------+------------+
|Sports|     5314.21|
+------+------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [8]:
new_films_df = (
    film_df.select("film_id", "title")
    .join(inventory_df.select("film_id"), on="film_id", how="left_anti")
    .drop("film_id")
    .orderBy("title")
)

new_films_df.show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [9]:
children_category_df = category_df.filter("name == 'Children'")

top_children_actor_df = (
    film_actor_df.select("actor_id", "film_id")
    .join(film_category_df.select("film_id", "category_id"), on="film_id")
    .join(F.broadcast(children_category_df.select("category_id", "name")), on="category_id")
    .join(F.broadcast(actor_df.select("actor_id", "first_name", "last_name")), on="actor_id")
    .groupBy("first_name", "last_name")
    .agg(F.count("film_id").alias("film_count"))
    .orderBy(F.col("film_count").desc())
    .limit(3)
)

top_children_actor_df.show()

+----------+---------+----------+
|first_name|last_name|film_count|
+----------+---------+----------+
|     HELEN|   VOIGHT|         7|
|     SUSAN|    DAVIS|         6|
|     RALPH|     CRUZ|         5|
+----------+---------+----------+



Stop Spark session:

In [10]:
spark.stop()